In [1]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker
import matplotlib.colors
import scipy.stats
import pandas
import itertools

mp.rcParams.update({'mathtext.default': 'regular'})
% matplotlib inline

In [2]:
working_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/'
#save_dir = '/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/calculations/npy_files/'
#latlon_indices = numpy.load(working_dir + 'ccal_latlon_indices_array.npy'); region='ccal'
#latlon_indices = numpy.load(working_dir + 'ncal_latlon_indices_array.npy'); region='ncal'
#latlon_indices = numpy.load(working_dir + 'scal_latlon_indices_array.npy'); region='scal'

PRECT_lat = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lat.npy')
PRECT_lon = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lon.npy')

In [3]:
working_dir = '/Users/baird/google_drive/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/'
#threshold=0.0
threshold=0.1
#threshold=0.5
#threshold=1.0
#threshold=5.0
#threshold=10.0

In [4]:
PRECT_nlat = 26
PRECT_nlon = 25

latlon_indices = list(itertools.product(range(PRECT_nlat), range(PRECT_nlon)))
region = 'whole_domain'
window=1

# Open preindustrial control data

In [5]:
year_start_pic = 402 #time_subsets[chunk,0]
year_end_pic = 2200 #time_subsets[chunk,1]

# create season strings
years_pic = numpy.arange(year_start_pic, year_end_pic+1, 1).astype(numpy.int)
half_years_pic = numpy.arange(year_start_pic+0.75, year_end_pic, 1)
#season_strings_pic = numpy.empty(years.size-1, dtype=numpy.str)

season_strings_pic = [str(years_pic[i])+'-'+str(years_pic[i+1]) for i in range(years_pic.size-1)]
member_strings_pic = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_pic=year_end_pic-year_start_pic

# Contruct extreme value distributions for each grid point

In [6]:
working_dir = '/Users/baird/Dropbox/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/whole_domain/'
# for each grid point:
# cycle through all times, get extreme 40d sum in each time period, store in array with length(season_strings_pic)
# once this is done, calculate the return periods of each of these
extreme_values_seasonal = numpy.zeros(( PRECT_nlat*PRECT_nlon, len(season_strings_pic) ))
for latlon_idx in range(len(latlon_indices)):
    if latlon_idx%10==0:
        print(latlon_idx)
    filename = 'member_005_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_'+'{:04d}'.format(year_start_pic)+'-'+'{:04d}'.format(year_end_pic)+'_threshold_'+str(threshold)+'mmday_'+region+'.npy'
    dict_pic = numpy.load(working_dir + filename).item()
    seasonal_events_list = [dict_pic[i]['seasonal_total'] for i in dict_pic.keys()]
    for season_idx in range(len(season_strings_pic)):
        extreme_values_seasonal[latlon_idx, season_idx] = numpy.nanmax(seasonal_events_list[season_idx])

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640


In [ ]:
extreme_values_seasonal_df = pandas.DataFrame(extreme_values_seasonal, columns=[season_strings_pic])
extreme_values_seasonal_df.to_csv('csv_files/extreme_values_seasonal_df.csv')

In [ ]:
extreme_values_seasonal_df = pandas.read_csv('csv_files/extreme_values_seasonal_df.csv')

# Open historical and RCP8.5 data

In [6]:
year_start = 1920
year_end = 2100

year_start_list = [1920,1950,1980,2010,2040,2070]
year_end_list = [1950,1980,2010,2040,2070,2100]

# create season strings
years = numpy.arange(year_start, year_end+1, 1).astype(numpy.int)
half_years_hist_rcp = numpy.arange(year_start+0.75, year_end, 1)

season_strings_hist_rcp = [str(years[i])+'-'+str(years[i+1]) for i in range(years.size-1)]
member_strings_hist_rcp = ['{:03d}'.format(i) for i in range(1,36)]

n_seasons_hist_rcp=year_end-year_start

In [7]:
ensemble_members = numpy.hstack((numpy.arange(1,36), numpy.arange(101,106)))
ensemble_names = ['{:03d}'.format(i) for i in ensemble_members]

In [ ]:
working_dir = '/Users/baird/Dropbox/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/whole_domain/'
# for each grid point:
# cycle through all times, get extreme 40d sum in each time period, store in array with length(season_strings_pic)
# once this is done, calculate the return periods of each of these
extreme_values_40d = numpy.zeros(( PRECT_nlat*PRECT_nlon, len(season_strings_hist_rcp) ))

for ensemble_idx in range(len(ensemble_members)):
    print(ensemble_names[ensemble_idx])
    for latlon_idx in range(len(latlon_indices)):
        filename = 'member_'+ensemble_names[ensemble_idx]+'_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_1920-2100_threshold_'+str(threshold)+'mmday_'+region+'.npy'
        #if latlon_idx%10==0:
        #    print(latlon_idx)
        dict_hist_rcp = numpy.load(working_dir + filename).item()
        sum_40d_list = [dict_hist_rcp[i]['running_40d_sum'] for i in dict_hist_rcp.keys()]
        for season_idx in range(len(season_strings_hist_rcp)):
            extreme_values_40d[latlon_idx, season_idx] = numpy.nanmax(sum_40d_list[season_idx])

    extreme_values_40d_df = pandas.DataFrame(extreme_values_40d, columns=[season_strings_hist_rcp])
    csv_filename = 'extreme_values_40d_sums_dataframe_model_'+ensemble_names[ensemble_idx]+'_1920-2100.csv'
    extreme_values_40d_df.to_csv('csv_files/'+csv_filename)
    print(csv_filename, 'written!')

001
